# GBP/USD Exchange Rate during 2017 UK Election Night - Under Construction



In [191]:
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
import statsmodels.api as sm

import pandas as pd
from collections import OrderedDict
from datetime import date

from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pylab

import tweepy

import json

import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [151]:
CONSUMER_KEY = '4BByuBKYk19fpSl5iMIkju3c0'
CONSUMER_SECRET = '2EK91aT0s7uMJ1oWECBRUwkXrxGykigrsmtqtOIAvFBPXiucQq'
ACCESS_TOKEN = '892729320736739328-E30nIY5dacqxeugxPoe3TXB2fIjITZB'
ACCESS_TOKEN_SECRET = 'WMViNA7y1d1trkb5nt7L5dOAHdScmYBMHm33sLeUVZrWT'


auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

tweets_raw = api.user_timeline(screen_name = 'bbcelection', count = 200, include_rts = False)
for i in range(0,5):
    oldest=tweets_raw[-1].id
    new_tweets = api.user_timeline(screen_name = 'bbcelection',count=200,max_id=oldest)
    tweets_raw.extend(new_tweets)

data = [[tw.created_at.year, tw.created_at.month, tw.created_at.day,"%s.%s"%(tw.created_at.hour, tw.created_at.minute), tw.id_str, tw.text.encode("utf8")] for tw in tweets_raw ]
tweets=pd.DataFrame(data, columns=['year','month','date','time','tweet_id','tweet'])


In [246]:
# Wikipedia data
UKpoll = pd.read_csv('data/UK2017Poll.txt', sep='\t', header=0)
UKpoll.columns=['Con_poll', 'Lab_poll', 'Lib_poll','SNP_poll','Pla_poll','Greens_poll',\
                'UKIP_poll', 'Other_poll', 'Seat','Region','2015']
results = pd.read_csv("data/result.csv")

In [152]:
tweets.tweet=tweets.tweet.astype(str)
tweets_cleaned = tweets[tweets.tweet.str.contains('#GE2017')]
tweets_cleaned['time_full'] = tweets_cleaned["year"].map(str)+ "/"+ tweets_cleaned["month"].map(str) + \
    "/"+ tweets_cleaned["date"].map(str)  + "/"+ tweets_cleaned["time"].map(str) 

C:\Users\AlexH\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Merging result data with time

In [247]:
results['time']=np.nan
for i in range(len(results)):
    tw = tweets_cleaned.tweet.str.contains(str(results.Constituency[i]))

    if True not in tw.tolist():
        
        #finds closes match
        max_score=0 
        for j in range(len(tweets_cleaned)):        
            score = fuzz.token_set_ratio(str(results.Constituency[i]),str(tweets_cleaned.tweet[j]))
            if score > max_score:
                tw_ind = j
                max_score=score
    else:
        tw_ind = tw.tolist().index(True)

        
    
    results['time'].loc[i]= datetime.strptime(tweets_cleaned.time_full[i],'%Y/%m/%d/%H.%M')

C:\Users\AlexH\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\AlexH\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


## Merging result data with time

In [270]:
full_table = pd.merge(results, UKpoll,  how='left', left_on='Constituency', right_on = 'Seat')
full_table = full_table.sort_values(by='time').reset_index(drop=True)

Constituency                Ynys Mon
Cnty                             GWN
Rgn                              WLS
Last Election                    Lab
Party                            Lab
Votes                         15,643
Share                         41.90%
Majrty                         5,259
turnout                       70.60%
Con[b]                        10,384
Lab[c]                        15,643
LD                               479
SNP                              NaN
UKIP                             624
Grn[d]                           NaN
DUP                              NaN
SF                               NaN
PC                            10,237
SDLP                             NaN
UUP                              NaN
APNI                             NaN
Other[e]                         NaN
Total                         37,367
time             2017-06-09 00:58:00
Con_poll                         NaN
Lab_poll                         NaN
Lib_poll                         NaN
S

## Analysis

In [286]:
columns = ['time', 'state','low', 'ave', 'high']
index= full_table.index
pred=pd.DataFrame(index=index, columns=columns)

sim_num = 20
for i in range(4,len(full_table)-7):
    
    #Trump regression
    #fit linear regression
    
    X_lab=np.asarray(full_table['Lab_poll'][0:i])
    Y_lab=np.asarray(full_table['Lab[c]'][0:i])
    X_lab=sm.add_constant(X_lab)
    model_lab = sm.RLM(Y_lab, X_lab,missing = 'drop').fit()
    
    #extract parameters
    const_lab = np.random.normal(model_lab.params[0], model_lab.bse[0], sim_num)
    slope_lab = np.random.normal(model_lab.params[1], model_lab.bse[1], sim_num)
    
    #Conservatives regression
    X_con=full_table['con_poll'][0:i]
    Y_con=full_table['con[b]'][0:i]
    X_con=sm.add_constant(X_con)
    model_con = sm.RLM(Y_con, X_con).fit()
    
    #extract parameters
    const_con = np.random.normal(model_con.params[0], model_con.bse[0], sim_num)
    slope_con = np.random.normal(model_con.params[1], model_con.bse[1], sim_num)
    

    
    result = np.zeros([sim_num, len(remain)])
    
    for j in range(0, len(const_dt)):
        
        #Liberal
        error_lab = np.random.normal(np.mean(model_lab.resid), np.std(model_lab.resid), sim_num)
        result_lab = np.array([v + const_lab[j] + slope_lab[j]*np.array(full_table['Lab_poll'][i:]) for v in error_lab])
        
        #Conservative
        error_con = np.random.normal(np.mean(model_con.resid), np.std(model_con.resid), sim_num)
        result_con = np.array([v + const_con[j] + slope_con[j]*np.array(full_table['con_poll'][i:]) for v in error_con])
        
        
        result_temp = result_lab>result_con
        result = np.append(result, result_temp,axis=0)
       
    #remove the zeros during initialisation
    result = np.delete(result,range(0,sim_num),0)
    result = result.astype(int)
        
    #filling the result of intermediates states with current count
#    select.result[select.result.isnull()[0:i]]=(select['trump'][select.result.isnull()[0:i]]>select['clinton'][select.result.isnull()[0:i]]).astype(int)
    
    EEV=sum(full_table[0:i])+ np.dot(result, remain['EV'])
#    print(jointb['time'][i],jointb['State'][i], EEV)

    #make table for plot
    pred.time[i]=datetime.strptime(full_table.time[i], '%Y-%m-%d %H:%M')
    pred.Constituency[i]=full_table.Constituency[i]
    pred.low[i]=np.mean(EEV)-np.std(EEV)
    pred.ave[i]=np.mean(EEV)
    pred.high[i]=np.mean(EEV)+np.std(EEV)

# keep only the last value at a certain time. And remove NAs.
pred=pred.drop_duplicates(subset='time', keep='last').dropna(axis=0, how='all')
pred.index = np.arange(0,len(pred))

TypeError: can't multiply sequence by non-int of type 'float'

In [287]:
X_lab

array([[  1.,  29.],
       [  1.,  46.],
       [  1.,  nan],
       [  1.,  14.]])